In [122]:
import glob
from find_source import make_catalog, combine_catalogs
import pandas as pd
import os
import astropy.units as u
from astropy.coordinates import SkyCoord
import json

In [123]:
def low_level_csv(folder, csv_path = './low_level.csv'):

    master_catalog = None
    old_df = None
    obs_id = 'Not Found'

    try:
        old_df = pd.read_csv(csv_path)
        master_catalog = (old_df.T).to_dict()
        mode = 'a'
        header = False
    except pd.errors.EmptyDataError: #if the file has no header and no data
        mode = 'w'
        header = True
    except FileNotFoundError: #file not found
        mode = 'w'
        header = True

    try:
        json_file = os.path.join(folder, 'polaris.json')
        with open(json_file, 'r') as file:
            obs_dict = json.load(file)

            #cleaning up obs_dict
            for key, value in obs_dict.items():
                if type(value) == list:
                    string = ', '.join(value)
                    obs_dict[key] = [string]
            obs_id = obs_dict.pop('obsID')
        if old_df is not None:
            old_df = old_df[old_df['Obs ID'] != obs_id] #removing old or outdated entries
    except Exception as e:
        print(f'Error with obsID: {e}. WARNING: Old/outdated data may not be deleted.')

    for file in glob.glob(os.path.join(folder, '*.fits')):
        try:
            catalog = make_catalog(file)
            for value in catalog.values():
                value['Obs ID'] = obs_id
                value['Source ID'] = None
            if master_catalog is None:
                master_catalog = catalog
            elif catalog is not None:
                master_catalog = combine_catalogs(master_catalog, catalog)
        except Exception as e:
            print(f'Error for {file}: {e}')

    df = pd.DataFrame.from_dict(master_catalog)
    df = df.T
    df.to_csv(csv_path, mode=mode, header=header, index=False)

In [124]:
def same_source(df, index1, index2):

    coord1 = SkyCoord(df['Coord RA'].iloc[index1], df['Coord Dec'].iloc[index1])
    coord2 = SkyCoord(df['Coord RA'].iloc[index2], df['Coord Dec'].iloc[index2])
    sep = coord1.separation(coord2)
    fwhm1 = float(df['Beam Maj Axis'].iloc[index1].replace(' arcsec', ''))
    fwhm2 = float(df['Beam Maj Axis'].iloc[index2].replace(' arcsec', ''))
    max_sep = (fwhm1 * fwhm2)**(1/2) * u.arcsec
    if sep <= max_sep:
        return True
    else:
        return False

In [125]:
def high_level_csv(low_level_path = './low_level.csv', high_level_path = './high_level.csv'):

    low_df = pd.read_csv(low_level_path)
    fields = low_df['Field Name']
    n_rows = len(fields)
    unique_fields = {}

    for i in range(n_rows):
        name = (fields[i]).lower() #lower to prevent issues with case sensitivity
        if name not in unique_fields:
            unique_fields[name] = [i]
        else:
            unique_fields[name].append(i)

    try:
        high_df = pd.read_csv(high_level_path)
        source_catalog = (high_df.T).to_dict(orient='list')
        for key in source_catalog.keys():
            if type(key) != str:
                del source_catalog[key]
    except Exception:
        source_catalog = {}

    for key, value in unique_fields.items():
        for i in value:
            source_count = 1
            source_name = f'{key}_{source_count}'
            added = False
            while (source_name in source_catalog) and (not added):
                for j in source_catalog[source_name]:
                    if i == j:
                        added = True
                        break
                    elif same_source(df=low_df, index1=i, index2=j):
                        source_catalog[source_name].append(i)
                        added = True
                        break
                source_count += 1
                source_name = f'{key}_{source_count}'
            if not added:
                source_catalog[source_name] = [i]

    max_length = max(len(row_list) for row_list in source_catalog.values())
    for row_list in source_catalog.values():
        row_list += [None] * (max_length - len(row_list))

    new_high_df = pd.DataFrame.from_dict(source_catalog)
    new_high_df = new_high_df.T
    new_high_df.to_csv(high_level_path, mode='w', header=False, index=True, index_label=False)


In [126]:
#%load_ext line_profiler

In [ ]:
#%%time
#low_level_csv('../data/250611_03:56:34')

Error for ../data/250611_03:56:34/1743-038_full.fits: "Keyword 'BMAJ' not found."
CPU times: user 7.85 s, sys: 246 ms, total: 8.09 s
Wall time: 8.78 s


In [128]:
#%lprun -f make_catalog low_level_csv('../data/multi_track')

In [129]:
#%lprun -f low_level_csv low_level_csv('../data/multi_track')

In [130]:
#%lprun -f high_level_csv high_level_csv()